In [ ]:
import pandas as pd
from google.colab import files

# Bước 1: Tải và đọc tệp Excel lên Colab
uploaded = files.upload()

# Đọc tệp Excel
file_name = list(uploaded.keys())[0]
data = pd.read_excel(file_name)

# Lấy tên các sheet từ cột thứ 3 (index 2)
sheet_names = data.iloc[:, 2].astype(str).values  # Đảm bảo là kiểu string

# Bỏ 3 cột đầu tiên nếu cần thiết
data = data.iloc[:, 3:]

# Bước 2: Định nghĩa các quy tắc phân chia cột thành hàng mới
breakdown_rules = [
    {"start_col": 0, "end_col": 68, "chunk_size": 2}, 
    {"start_col": 68, "end_col": 104, "chunk_size": 3},  
    {"start_col": 104, "end_col": None, "chunk_size": 2}  
]

# Bước 3: Xử lý dữ liệu theo các quy tắc đã định nghĩa
sheets = {}

for index, row in data.iterrows():
    # Lấy tên sheet từ sheet_names hoặc fallback nếu thiếu
    sheet_name = sheet_names[index] if index < len(sheet_names) else f"Pattern {index + 1}"

    # Tạo danh sách để lưu các hàng mới
    new_rows = []

    for rule in breakdown_rules:
        start_col = rule["start_col"]
        end_col = rule["end_col"]
        chunk_size = rule["chunk_size"]

        # Lấy các cột theo quy tắc
        if end_col is not None:
            cols = row[start_col:end_col]
        else:
            cols = row[start_col:]

        # Chia cột thành các chunk_size
        for i in range(0, len(cols), chunk_size):
            chunk = cols[i:i + chunk_size].values
            # Kiểm tra xem chunk có chứa dữ liệu không (không phải toàn NaN)
            if pd.notna(chunk).any():
                row_index = f"Pattern {len(new_rows) + 1}"
                # Thêm dữ liệu vào hàng mới
                new_row = [row_index] + list(chunk)
                new_rows.append(new_row)

    # Nếu có hàng mới, thêm vào dictionary sheets
    if new_rows:
        # Xác định số lượng cột tối đa trong các hàng mới
        max_columns = max(len(r) for r in new_rows)
        # Tạo tên cột cho DataFrame
        columns = ["Pattern"] + [f"Col{i+1}" for i in range(max_columns - 1)]
        # Tạo DataFrame cho sheet hiện tại
        sheet_df = pd.DataFrame(new_rows, columns=columns)
        sheets[sheet_name] = sheet_df

# Bước 4: Xuất kết quả ra tệp Excel với nhiều sheet
output_file_name = "processed_output_sheets.xlsx"

with pd.ExcelWriter(output_file_name, engine='xlsxwriter') as writer:
    for sheet_name, sheet_data in sheets.items():
        # Đảm bảo tên sheet hợp lệ theo quy định của Excel
        safe_sheet_name = str(sheet_name)[:31].replace(':', '').replace('\\', '').replace('/', '').replace('*', '').replace('?', '').replace('[', '').replace(']', '')
        sheet_data.to_excel(writer, sheet_name=safe_sheet_name, index=False)

# Bước 5: Tải tệp kết quả về máy
files.download(output_file_name)
